In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(credentials)
sheet1 = client.open("NBA Bets 2021-22").get_worksheet(0)

In [2]:
import pandas as pd

In [3]:
mdf = pd.DataFrame(sheet1.get_all_records())

mdf

,participant full name,date,opponent,home,spread,moneyline,total,points scored,points allowed,total points scored,lost by,spread result,moneyline result,total result
0,Milwaukee Bucks,2021-10-19,Brooklyn Nets,TRUE,0.0,100,240.5,127,104,231,-23,win,win,loss
1,Brooklyn Nets,2021-10-19,Milwaukee Bucks,FALSE,0.0,-120,240.5,104,127,231,23,loss,loss,loss
2,L.A. Lakers,2021-10-19,Golden State Warriors,TRUE,-4.5,-190,230.5,114,121,235,7,loss,loss,win
3,Golden State Warriors,2021-10-19,L.A. Lakers,FALSE,4.5,160,230.5,121,114,235,-7,win,win,win
4,Charlotte Hornets,2021-10-20,Indiana Pacers,TRUE,-2.0,-130,228.5,123,122,245,-1,loss,win,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1277,Boston Celtics,2022-01-15,Chicago Bulls,TRUE,-5.0,-220,219.5,,,,,push,loss,push
1278,Denver Nuggets,2022-01-15,L.A. Lakers,TRUE,-3.5,-160,224.5,,,,,push,loss,push
1279,L.A. Lakers,2022-01-15,Denver Nuggets,FALSE,3.5,135,224.5,,,,,push,loss,push
1280,Dallas Mavericks,2022-01-15,Orlando Magic,TRUE,-10.5,-605,208.0,,,,,push,loss,push


In [4]:
mdf.dtypes


participant full name     object
date                      object
opponent                  object
home                      object
spread                   float64
moneyline                  int64
total                    float64
points scored             object
points allowed            object
total points scored       object
lost by                   object
spread result             object
moneyline result          object
total result              object
dtype: object

In [5]:
# I want the lakers' last 3 games record against the spread as a favorite

relevant_df = mdf.loc[(mdf['participant full name'] == 'L.A. Lakers') & (mdf['spread'] < 0)][-3:]

len(relevant_df.loc[relevant_df['spread result'] == 'win']), len(relevant_df.loc[relevant_df['spread result'] == 'loss']), len(relevant_df.loc[relevant_df['spread result'] == 'push'])

(1, 2, 0)

In [6]:
# get a team's record over last "games" amount of games (defaulted to all games) for specified bet (default to moneyline)

def team_record(dataframe, team, bet='moneyline result', games=82):
    spec_team = dataframe.loc[dataframe['participant full name'] == team][games*(-1):]
    return len(spec_team.loc[spec_team[bet] == 'win']), len(spec_team.loc[spec_team[bet] == 'loss']), len(spec_team.loc[spec_team[bet] == 'push'])

team_record(dataframe=mdf, team='L.A. Clippers', bet='total result')

(19, 24, 1)

In [7]:
def vs_opp(dataframe, team, opponent, bet='moneyline result', games=82):
    team_record(dataframe, team, bet, games)


In [8]:
def record_return(func, bet='moneyline result'):
    def wrapper_record_return(*args, **kwargs):
        spec_team = func(*args, **kwargs)
        return len(spec_team.loc[spec_team[bet] == 'win']), len(spec_team.loc[spec_team[bet] == 'loss']), len(spec_team.loc[spec_team[bet] == 'push'])
    return wrapper_record_return

def r(dataframe, team, games=82):
    return dataframe.loc[dataframe['participant full name'] == team][games*(-1):]

r(mdf, 'Golden State Warriors')

rr = record_return(r, 'spread')(mdf, 'Golden State Warriors')

rr

(0, 0, 0)

In [9]:
def f1(n):
    n += '1'
    return n

def f2(n):
    n += '2'
    return n

func_ls = [f1, f2]
a = 'a'
for func in func_ls:
    a = func(a)

a

'a12'

In [10]:
def team_func(dataframe, team):
    return dataframe.loc[dataframe['participant full name'] == team]

def opp_func(dataframe, opponent):
    return dataframe.loc[dataframe['opponent'] == opponent]

def last_games_func(dataframe, n):
    return dataframe.iloc[(-1 * n):, :]

def home_func(dataframe, home_or_not):
    return dataframe.loc[dataframe['home'] == home_or_not]

def spread_func(dataframe, spread_filt_settings: dict):
    d ={
        'exact': dataframe.loc[(dataframe['spread'] == spread_filt_settings['points'])],
        'longer': dataframe.loc[dataframe['spread'] >= spread_filt_settings['points']],
        'shorter': dataframe.loc[dataframe['spread'] <= spread_filt_settings['points']]
    }
    return d[spread_filt_settings['kind']]

def favorite_func(dataframe, favorite):
    if favorite:
        return dataframe.loc[dataframe['spread'] < 0]
    else:
        return dataframe.loc[dataframe['spread'] > 0]


In [17]:
filters = {
    'team': team_func,
    'opponent': opp_func,
    'home': home_func,
    'spread': spread_func,
    'favorite': favorite_func,
    'last n games': last_games_func
}

In [18]:
"""
filter_these = {
    'bet': "spread result" or "moneyline result" or "total result"
    'team': str,
    'opponent': str,
    'last n games': int,
    'home': "TRUE" or "FALSE",
    'spread': {
        'kind': "exact" or "longer" or "shorter",
        'points': float
        },
    'favorite': bool
}
"""

'\nfilter_these = {\n    \'bet\': "spread result" or "moneyline result" or "total result"\n    \'team\': str,\n    \'opponent\': str,\n    \'last n games\': int,\n    \'home\': "TRUE" or "FALSE",\n    \'spread\': {\n        \'kind\': "exact" or "longer" or "shorter",\n        \'points\': float\n        },\n    \'favorite\': bool\n}\n'

In [33]:
filter_these = {
    'bet': 'total result',
    'team': 'Minnesota Timberwolves',
    'last n games': 7
}

In [34]:
df = mdf

for key in filters:
    if key in filter_these:
        df = filters[key](df, filter_these[key])

print("Record is", len(df.loc[df[filter_these['bet']] == 'win']), "wins", len(df.loc[df[filter_these['bet']] == 'loss']), "losses", len(df.loc[df[filter_these['bet']] == 'push']), "pushes")

df

Record is 4 wins 3 losses 0 pushes


,participant full name,date,opponent,home,spread,moneyline,total,points scored,points allowed,total points scored,lost by,spread result,moneyline result,total result
1078,Minnesota Timberwolves,2022-01-02,L.A. Lakers,FALSE,7.5,275,223.5,103,108,211,5,win,loss,loss
1098,Minnesota Timberwolves,2022-01-03,L.A. Clippers,FALSE,3.0,130,219.0,122,104,226,-18,win,win,win
1122,Minnesota Timberwolves,2022-01-05,Oklahoma City Thunder,TRUE,-7.5,-380,213.5,98,90,188,-8,win,win,loss
1146,Minnesota Timberwolves,2022-01-07,Oklahoma City Thunder,FALSE,-5.5,-225,215.0,135,105,240,-30,win,win,win
1178,Minnesota Timberwolves,2022-01-09,Houston Rockets,FALSE,-5.5,-260,229.0,141,123,264,-18,win,win,win
1208,Minnesota Timberwolves,2022-01-11,New Orleans Pelicans,FALSE,-3.5,-160,227.0,125,128,253,3,loss,loss,win
1238,Minnesota Timberwolves,2022-01-13,Memphis Grizzlies,FALSE,4.5,190,229.5,108,116,224,8,loss,loss,loss


In [35]:
df['total points scored'].mean()

229.42857142857142